# Loading packages

In [1]:
import numpy as np
import pandas as pd
import glob

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
from pandas.api.types import CategoricalDtype

# Loading data

In [4]:
data= pd.read_pickle("../data/modified_exclusions/pt_replication_modified_exclusions_data.pkl")

In [5]:
data.head()

,1,2,3,4,5,6,7,8,9,10,...,Loss_Intuition,Country,Duration_in_seconds,Language,Sample,Duration_Minutes,YoB,Awareness_of_Loss_Aversion,Financial_Stress,Presentation_Order
0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,1.0,Bulgaria,356,Bulgarian,Direct,5.933333,1992,3,NaN,4|6|2|3|17|18|14|8|7|15|1|12|11|5|13|16|9|10
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,NaN,Bulgaria,931,Bulgarian,Direct,15.516667,1979,1,NaN,7|17|16|11|15|18|4|9|1|13|2|10|6|14|5|12|8|3
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,Bulgaria,462,Bulgarian,Direct,7.700000,1975,3,NaN,11|8|4|7|12|18|3|14|6|16|17|1|2|13|5|15|9|10
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,NaN,Bulgaria,412,Bulgarian,Direct,6.866667,1977,1,NaN,16|7|6|9|12|18|10|8|5|15|2|3|4|14|11|1|17|13
4,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,...,NaN,Bulgaria,490,Bulgarian,Direct,8.166667,1970,1,NaN,13|4|3|7|9|18|16|8|14|1|6|12|17|10|5|2|11|15


In [6]:
PT_contrasts_original_df = pd.read_excel("../output/original_contrasts.xlsx")
PT_contrasts_original_df.head()

,Effect,Item 1,Item 2,A1,B1,A2,B2,OR,p-value,OR2,Significance,LogOR
0,Certainty Effect,1,2,13,59,60,12,0.044068,1.558248e-15,22.692308,True,-3.122026
1,Certainty Effect,3,4,19,76,62,33,0.133065,3.373660e-10,7.515152,True,-2.016921
2,Certainty Effect,7,8,87,8,40,55,14.953125,1.598340e-13,14.953125,True,2.704920
3,Reflection Effect,3,7,19,76,87,8,0.022989,4.395039e-25,43.500000,True,-3.772761
4,Reflection Effect,4,8,62,33,40,55,2.583333,2.163003e-03,2.583333,True,0.949081


### Quantifying how many people changed their mind in the original experiment

In [7]:
PT_contrasts_original_df["A_difference"] = PT_contrasts_original_df["A1"] - PT_contrasts_original_df["A2"]

In [8]:
PT_contrasts_original_df[["Item 1", "Item 2", "A_difference"]]

,Item 1,Item 2,A_difference
0,1,2,-47
1,3,4,-43
2,7,8,47
3,3,7,-68
4,4,8,22
5,5,9,-52
6,6,10,28
7,16,17,40
8,4,11,31
9,5,6,-39


### Quantifying how many people changed their minds in the direction of Prospect Theory in the current study

In [11]:
contrast_cat = CategoricalDtype(categories=["EUT + PT", "PT",
                                            "Neither"],
                               ordered=True)
contrast_columns = []
for ix in PT_contrasts_original_df.index:
    i1 = PT_contrasts_original_df.loc[ix, "Item 1"]
    i2 = PT_contrasts_original_df.loc[ix, "Item 2"]
    i1 = str(i1)
    i2 = str(i2)
    diff = PT_contrasts_original_df.loc[ix, "A_difference"]
    column_title = "Contrast_" + i1 + '_' + i2
    data[column_title] = np.nan
    data.loc[(data[i1]==0) & (data[i2]==0), column_title] = "EUT + PT"
    data.loc[(data[i1]==1) & (data[i2]==1), column_title] = "EUT + PT"
    if diff > 0:
        data.loc[(data[i1]==1) & (data[i2]==0), column_title] = "PT"
        data.loc[(data[i1]==0) & (data[i2]==1), column_title] = "Neither"
    if diff < 0:
        data.loc[(data[i1]==0) & (data[i2]==1), column_title] = "PT"
        data.loc[(data[i1]==1) & (data[i2]==0), column_title] = "Neither"
    data[column_title] = data[column_title].astype(contrast_cat)
    contrast_columns.append(column_title)

In [13]:
data.columns

Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
       '14', '15', '16', '17', 'Gender', 'Age', 'Personal_Income',
       'Financial_Situation', 'Bill_Payments', 'Education', 'Credit_Debt',
       'Additional_Income_Private_Investment', 'Additional_Income_Property',
       'Additional_Income_Business_Ownership',
       'Additional_Income_Retirement_Plan', 'Loss_Awareness', 'Loss_Intuition',
       'Country', 'Duration_in_seconds', 'Language', 'Sample',
       'Duration_Minutes', 'YoB', 'Awareness_of_Loss_Aversion',
       'Financial_Stress', 'Presentation_Order', 'Contrast_1_2',
       'Contrast_3_4', 'Contrast_7_8', 'Contrast_3_7', 'Contrast_4_8',
       'Contrast_5_9', 'Contrast_6_10', 'Contrast_16_17', 'Contrast_4_11',
       'Contrast_5_6', 'Contrast_9_10', 'Contrast_12_13', 'Contrast_14_15'],
      dtype='object')

### Saving choice contrast data

In [35]:
data.to_excel("../output/pt_replication_modified_exclusions_data_CoM.xlsx", index=False)
data.to_pickle("../output/pt_replication_modified_exclusions_data_CoM.pkl")
data.to_csv("../output/pt_replication_modified_exclusions_data_CoM.csv", index=False)

### Displaying choice proportions per contrast

In [15]:
for column in contrast_columns:
    display(round(data[column].value_counts(sort=False, normalize=True), 2))

EUT + PT    0.48
PT          0.44
Neither     0.08
Name: Contrast_1_2, dtype: float64

EUT + PT    0.54
PT          0.42
Neither     0.04
Name: Contrast_3_4, dtype: float64

EUT + PT    0.54
PT          0.37
Neither     0.08
Name: Contrast_7_8, dtype: float64

EUT + PT    0.25
PT          0.71
Neither     0.04
Name: Contrast_3_7, dtype: float64

EUT + PT    0.46
PT          0.27
Neither     0.26
Name: Contrast_4_8, dtype: float64

...

EUT + PT    0.54
PT          0.41
Neither     0.06
Name: Contrast_4_11, dtype: float64

EUT + PT    0.40
PT          0.58
Neither     0.03
Name: Contrast_5_6, dtype: float64

EUT + PT    0.48
PT          0.44
Neither     0.08
Name: Contrast_9_10, dtype: float64

EUT + PT    0.48
PT          0.44
Neither     0.08
Name: Contrast_12_13, dtype: float64

EUT + PT    0.44
PT          0.49
Neither     0.07
Name: Contrast_14_15, dtype: float64

### Preparing a table for the figure

In [29]:
contrast_effect_df = data[contrast_columns].melt(var_name="Effect", value_name="Direction")
contrast_table = contrast_effect_df.pivot_table(index="Effect", columns="Direction", values="Direction", aggfunc=len)
contrast_table = (contrast_table.transpose()/contrast_table.sum(1)).transpose()*100
contrast_table = contrast_table.loc[contrast_columns[::-1], :]

### Contrast table summary statistics

In [23]:
contrast_table.mean()

Direction
EUT + PT    44.069942
Neither      9.453989
PT          46.476070
dtype: float64

In [24]:
contrast_table["PT"].max().round()

71.0

In [25]:
contrast_table["PT"].min().round()

27.0

In [26]:
contrast_table["Neither"].std()

7.5224556580294015

In [27]:
contrast_table["PT"].std()

12.903678216906268

### Saving the contrast table

In [31]:
contrast_table.to_excel("../output/contrast_changes_of_mind.xlsx")
contrast_table.to_csv("../output/contrast_changes_of_mind.csv")